In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr'

%load_ext autoreload
%autoreload 2

import sys
sys.path.append(r'c:\git\cameratraps')  # append this repo to PYTHONPATH

import json
import os
from collections import Counter, defaultdict
from random import sample
import math

from tqdm import tqdm

from data_management.megadb.schema import sequences_schema_check
from data_management.annotations.add_bounding_boxes_to_megadb import *
from data_management.megadb.converters.cct_to_megadb import make_cct_embedded, process_sequences, write_json

In [ ]:
path_to_output_public = r'F:\data_staging\island-conservation\island_conservation_megadb.json'
dataset_name = 'island_conservation'
label_map = {'human': 'person', 'vehicle': 'vehicle', 'empty': 'empty'}

# Path to the CCT json, or a loaded json object
path_to_image_cct = r"F:\data_staging\island-conservation\island_conservation.json"
path_to_bbox_cct = r"F:\data_staging\island-conservation\island_conservation.json"
assert not (path_to_image_cct is None and path_to_bbox_cct is None)

In [ ]:
embedded = make_cct_embedded(image_db=path_to_image_cct, bbox_db=path_to_bbox_cct)

In [ ]:
sequences = process_sequences(embedded, dataset_name)

In [ ]:
for index, entry in enumerate(sequences):
    for prop in ['id', 'bbox', 'class', 'conf']:
        if prop == 'conf' or prop == 'class':
            sequences[index]['images'][0][prop] = entry[prop]
        elif prop == 'bbox':
            for pos, ele in enumerate(entry[prop]):
                ele['class'] = ele['category']
                ele['category'] = label_map.get(ele['category'], 'animal')
                entry[prop][pos] = ele
            sequences[index]['images'][0][prop] = entry[prop]
        del entry[prop]
    sequences[index] = entry

In [ ]:
sequences_schema_check.sequences_schema_check(sequences)

In [ ]:
write_json(path_to_output_public, sequences)